# Chi-squared tests 

## Chi-squared tests for goodness of fit and tests of independence

Tests dealing with categorical data are based on variable counts and not the actual value of the variable being measured.

## Chi-squared goodness-of-fit test

### The Chi-squared goodness-of-fit test is an analog of the one-sample t-test. It tests to see whether the distribution of sample categorical data matches an expected distribution.

You could use the Chi-squared goodness-of-fit test to assess whether the demographics of a sample (e.g. school, town, region) differ/ or match that of the general population. 

In [1]:
# Starting with software library imports:

import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
# Generating some made-up example data for the demographics of a county compared to the rest of the UK, to use for analyses.

national = pd.DataFrame(["white"] * 100000 + ["medit"] * 60000 +\
                        ["black"] * 50000 + ["asian"] * 15000 + ["other"] * 35000)

yorkshire = pd.DataFrame(["white"] * 600 + ["medit"] * 300 +\
                        ["black"] * 250 + ["asian"] * 75 + ["other"] * 150)

national_table = pd.crosstab(index = national[0], columns = "count")
yorks_table = pd.crosstab(index = yorkshire[0], columns = "count")

print("National")
print(national_table)
print(" ")
print("Yorkshire")
print(yorks_table)

# This could be a break down of ethicity of voters nationally and in the county of Yorkshire. 
# The question we are addressing with a goodness-of-fit model is: are the counts/ distribution across categories the
# same for Yorkshire as the national population. 

National
col_0   count
0            
asian   15000
black   50000
medit   60000
other   35000
white  100000
 
Yorkshire
col_0  count
0           
asian     75
black    250
medit    300
other    150
white    600


Chi-squared analysis is based on the chi-squared statistic. 

This is calculated by: X^2 = sum((observed - expected)^2 / expected)

In the formula, observed is the observed counts for each category, expected is expected count based on the distribution of the population for the corresponding category. 

In [3]:
# Here is a manual way to calculate the chi-squared statistic for this dataset:

observed = yorks_table

national_ratios = national_table / len(national) # Get the population ratios. 

print(national_ratios)

expected = national_ratios * len(yorkshire)  # Gives us the expected counts. 

chi_squared_stat = (((observed - expected)**2)/ expected).sum()
print('------')
print(chi_squared_stat)

col_0     count
0              
asian  0.057692
black  0.192308
medit  0.230769
other  0.134615
white  0.384615
------
col_0
count    18.194805
dtype: float64


## To assess whether our Chi-squared test statistic is significant we have to compare it to a critical value based on the Chi-squared distribution.

### Note: The Chi-squared test assumes that the expected counts are five or more in 80% of cells and no cell should have an expected count of less than one.

#### We can use the scipy library to calculate our critical value and the p-value for this test.

In [4]:
# Calculating the critical value using a 95% confidence level:

crit = stats.chi2.ppf(q = 0.95, df = 4) # Find the critical value for 95% CI's and df is number of categories - 1. 
# Note: can use quantile of 0.95 as chi test is one-tailed. 
print("Critical Value:")
print(crit)

p_value = 1 - stats.chi2.cdf(x = chi_squared_stat, df = 4) # Find the p-value. 

print("P value:")
print(p_value)

Critical Value:
9.487729036781154
P value:
[0.00113047]


Since the Chi-squared statistic above exceeds the critical value we can reject the null hypothesis that the two distributions are the same and accept the alternative hypothesis that they are different and the distribution of ethnicity in Yorkshire differs from the distribution nationally. 

#### We can carry out a Chi-squared goodness-of-fit test automatically using the scipy function scipy.stats.chisquare():

In [5]:
stats.chisquare(f_obs = observed, f_exp = expected) # These are our arrays of observed and expected counts. 


Power_divergenceResult(statistic=array([18.19480519]), pvalue=array([0.00113047]))

The test results agree with the results of the Chi-squared test we conducted manually. 

# Chi-squared test of Independence

Independence is a key concept in probability where knowing the value of one variable tells you nothing about the values of another variable. The Chi-squared test of independence tells you whether two variables are independent of each other. 

If two variables are significantly associated with each other (we get a significant Chi-square result) then they are not independent. For example, we may want to assess whether political affiliations vary based on demographic factors, such as ethicnicty or sex. If we get a significant Chi-square result, then these variables are significantly associated and not independent of each other.    

In [6]:
# Generating some simulated voter demographic information to assess independence using Chi-squared. 

np.random.seed(10)

# sample data randomly at fixed probabilities

voter_ethnic = np.random.choice(a = ["asian", "black", "medit", "other", "white"],
                               p = [0.05, 0.15, 0.25, 0.05, 0.5], 
                               size = 1000)

# sample data randomly at fixed probabilities

voter_party = np.random.choice(a = ["labour", "liberal", "conservative"], 
                              p = [0.4, 0.2, 0.4], 
                              size = 1000)

voters = pd.DataFrame({"Ethnicity": voter_ethnic, "Party": voter_party})

voter_tab = pd.crosstab(voters.Ethnicity, voters.Party, margins = True)

voter_tab.columns = ["labour", "liberal", "conservative", "row_total"]
voter_tab.index = ["asian", "black", "medit", "other", "white", "col_total"]

observed = voter_tab.iloc[0:5, 0:3] # Gives the cross tab without row and column totals (used later). 

voter_tab

,labour,liberal,conservative,row_total
asian,32,21,7,60
black,64,65,25,154
medit,94,107,50,251
other,15,15,8,38
white,212,189,96,497
col_total,417,397,186,1000


For a Chi-squared test of independence we use the same formula that we used for a goodness-of-fit test. The main difference is that we calculate the expected counts in a 2-dimensional table instead of a 1-dimensional table. 
To get the expected count for a cell we multiply the row total by the column total and then divide by the total number of observations. 
We can get the expected counts for all cells in a table by taking the row and column totals, performing an outer product on them with the np.outer() function and dividing by the number of observations.  

In [7]:
# Calculating expected counts and creating an new dataframe showing those expected counts:

expected = np.outer(voter_tab["row_total"][0:5], voter_tab.loc["col_total"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["labour", "liberal", "conservative"]
expected.index = ["asian", "black", "medit", "other", "white"]

expected

,labour,liberal,conservative
asian,25.020,23.820,11.160
black,64.218,61.138,28.644
medit,104.667,99.647,46.686
other,15.846,15.086,7.068
white,207.249,197.309,92.442


In [8]:
# Calculating the Chi-squared statistic:

chi_square_stat = (((observed - expected)**2)/ expected).sum().sum() # As this is a 2-dimensional table we run .sum() twice.
# This gives us the column sums the first time and then adds the column sums the second time. 

print(chi_square_stat)

7.169321280162059


In [9]:
crit = stats.chi2.ppf(q = 0.95, df = 8) # Find the critical value for 95% CI. 
# df for a 2-dimensional contingency table is calculated as: df = (rows-1)*(cols-1)

print("Critical Value")
print(crit)

p_value = 1 - stats.chi2.cdf(x = chi_square_stat, df = 8)

print("P Value")
print(p_value)

Critical Value
15.50731305586545
P Value
0.518479392948842


We can also run a Chi-squared test of independence using scipy as shown below:

In [10]:
# Here we only need to pass in our table of observed counts and scipy will work out the expected counts and conduct the test.

stats.chi2_contingency(observed = observed)

# Note the output gives us our Chi-squared test statistic, the p-vlaue and an array of expected counts.

(7.169321280162059,
 0.518479392948842,
 8,
 array([[ 25.02 ,  23.82 ,  11.16 ],
        [ 64.218,  61.138,  28.644],
        [104.667,  99.647,  46.686],
        [ 15.846,  15.086,   7.068],
        [207.249, 197.309,  92.442]]))

The above result matches our results obtained manually.

### In summary, the non-significant test indicates that the variables are not associated and we fail to reject the null hypothesis that they are independent.

# Chi-squared test of independence on a real dataset. 



In [11]:
# Importing the health dataset to use for analysis:

health = pd.read_csv("health.csv")

health.head()

,timedrs,phyheal,menheal,stress,religion,race
0,1,5,8,265,3,1
1,3,4,6,415,3,1
2,0,3,4,92,2,1
3,13,2,2,241,1,1
4,15,3,6,86,4,1


In [12]:
health.dtypes

timedrs     int64
phyheal     int64
menheal     int64
stress      int64
religion    int64
race        int64
dtype: object

In [13]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   timedrs   465 non-null    int64
 1   phyheal   465 non-null    int64
 2   menheal   465 non-null    int64
 3   stress    465 non-null    int64
 4   religion  465 non-null    int64
 5   race      465 non-null    int64
dtypes: int64(6)
memory usage: 21.9 KB


In [14]:
# Converting religion to a categorical variable. 
# Creating labels for integer values:

a = [1, 2, 3, 4]
b = ['other', 'catholic', 'protestant', 'jewish']

health['religion_'] = health['religion'].map(dict(zip(a, b)))

In [15]:
health.head()

,timedrs,phyheal,menheal,stress,religion,race,religion_
0,1,5,8,265,3,1,protestant
1,3,4,6,415,3,1,protestant
2,0,3,4,92,2,1,catholic
3,13,2,2,241,1,1,other
4,15,3,6,86,4,1,jewish


In [16]:
# Converting race to a categorical variable.
# Creating labels for integer values:

c = [1, 2]
l = ['white', 'non_white']

health['race'] = health['race'].map(dict(zip(c, l)))

health.head()

,timedrs,phyheal,menheal,stress,religion,race,religion_
0,1,5,8,265,3,white,protestant
1,3,4,6,415,3,white,protestant
2,0,3,4,92,2,white,catholic
3,13,2,2,241,1,white,other
4,15,3,6,86,4,white,jewish


In [17]:
health.dtypes

# Race and religion_ now shown as objects

timedrs       int64
phyheal       int64
menheal       int64
stress        int64
religion      int64
race         object
religion_    object
dtype: object

In [18]:
health_tab = pd.crosstab(health['religion_'], health['race'])

health_tab

race,non_white,white
religion_,,
catholic,25,94
jewish,3,89
other,6,70
protestant,6,169


In [19]:
# Running chi-square on the above contingency table

chival, pval, df, exp = stats.chi2_contingency(observed = health_tab)

chival, pval, df, exp

# The output gives us the chi-squared statistic, the p-value, the df, and the array of expected counts. 
# Here we can see we have a significant association between the variables. 

(32.448064911174924,
 4.2105695778398946e-07,
 3,
 array([[ 10.3030303 , 108.6969697 ],
        [  7.96536797,  84.03463203],
        [  6.58008658,  69.41991342],
        [ 15.15151515, 159.84848485]]))

### Following up a significant chi-square test with post-hoc tests. 

A significant Chi-square test of independence tells us the variables are not independent and are significantly associated, but how do we interpret where that significant association lies? Eyeballing contingency tables can be misleading and what look like large differences between observed and expected counts can be hard to interpret as to whether those differences are statistically significant. 

To interpret the significant association it is useful to use standardised residual and adjusted standardised residuals. 

There is no existing package to conduct this type of post-hoc analysis so we have to code it ourselves:

In [20]:
# To do this we need to obtain the row totals and column totals for our contingency table. We also need the number of rows and
# number of columns. 

col_totals = health_tab.sum()

n_cols = len(col_totals)

n_cols, col_totals

(2,
 race
 non_white     40
 white        422
 dtype: int64)

In [21]:
# Now the row totals and counts:

row_totals = health_tab.sum(axis = 1)

n_rows = len(row_totals)

n_rows, row_totals

(4,
 religion_
 catholic      119
 jewish         92
 other          76
 protestant    175
 dtype: int64)

In [22]:
# It is also useful to have the grand total (N). This can be obtained just by summing the row or the column counts:

n = sum(row_totals)

n

462

We now have everything to fill out the adjusted residuals formula as shown below:

In [23]:
for i in range(n_rows):
    for j in range(n_cols):
        adjres = (health_tab.iloc[i, j] - exp[i, j]) / (exp[i, j] * (1 - row_totals[i]/n)*(1 - col_totals[j]/n))**0.5
        print(adjres)

5.560117893362686
-5.56011789336269
-2.0569957978427866
2.0569957978427844
-0.25886248275603163
0.2588624827560345
-3.1211150119859656
3.1211150119859683


In [24]:
# The above gave us the adjusted residuals but it would be useful to know what cell these residuals relate to. 
# We can achieve this by creating a dataframe as follows:

health_res = pd.DataFrame(columns = ['religion', 'race', 'adj. res.'])
for i in range(n_rows):
    for j in range(n_cols):
        adjres = (health_tab.iloc[i, j] - exp[i, j]) / (exp[i, j] * (1 - row_totals[i]/n)*(1 - col_totals[j]/n))**0.5
        health_res = health_res.append({'religion': health_tab.index[i], 'race': health_tab.columns[j], 'adj. res.': adjres}, ignore_index = True)

health_res

C:\Users\robert65\AppData\Local\Temp\ipykernel_13116\4266321650.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  health_res = health_res.append({'religion': health_tab.index[i], 'race': health_tab.columns[j], 'adj. res.': adjres}, ignore_index = True)
C:\Users\robert65\AppData\Local\Temp\ipykernel_13116\4266321650.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  health_res = health_res.append({'religion': health_tab.index[i], 'race': health_tab.columns[j], 'adj. res.': adjres}, ignore_index = True)
C:\Users\robert65\AppData\Local\Temp\ipykernel_13116\4266321650.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  health_res = health_res.append({'religion': health_tab.index[i], 'race': health_tab.columns[j], 'adj. res.'

,religion,race,adj. res.
0,catholic,non_white,5.560118
1,catholic,white,-5.560118
2,jewish,non_white,-2.056996
3,jewish,white,2.056996
4,other,non_white,-0.258862
5,other,white,0.258862
6,protestant,non_white,-3.121115
7,protestant,white,3.121115


The above adjusted residuals are z values and can be used to interpret significant deviations from a mean of zero for the standard normal distribution. 

We can convert them to their corresponding probabilities (p-values) for the standard normal distribution using the norm.cdf function from scipy.stats: 

In [25]:
from scipy.stats import norm

In [26]:
ar_sig = np.absolute(health_res['adj. res.'])

ar_sig

# Note here I have obtained the absolute value of adjusted residuals but they are an object and not a float

0    5.560118
1    5.560118
2    2.056996
3    2.056996
4    0.258862
5    0.258862
6    3.121115
7    3.121115
Name: adj. res., dtype: object

In [27]:
# Converting them to a float using the pandas to_numeric method. 
ar_abs = pd.to_numeric(ar_sig, downcast = 'float')

ar_abs

0    5.560118
1    5.560118
2    2.056996
3    2.056996
4    0.258862
5    0.258862
6    3.121115
7    3.121115
Name: adj. res., dtype: float32

In [28]:
# The above ar_abs float values can now be used in the formula to find the respective p-values. 
health_res['sig.'] = 2 * (1 - norm.cdf(ar_abs))
health_res

# Note above we are mutliplying by two because we want two-tailed sig. 

,religion,race,adj. res.,sig.
0,catholic,non_white,5.560118,2.695927e-08
1,catholic,white,-5.560118,2.695927e-08
2,jewish,non_white,-2.056996,3.968662e-02
3,jewish,white,2.056996,3.968662e-02
4,other,non_white,-0.258862,7.957413e-01
5,other,white,0.258862,7.957413e-01
6,protestant,non_white,-3.121115,1.801677e-03
7,protestant,white,3.121115,1.801677e-03


The above significance values have been appended to the table. However, we are doing multiple tests, so we need some type of adjustment. Using the Bonferroni adjustment to correct for the fact we have done 8 tests. 

In [29]:
health_res.dtypes

religion      object
race          object
adj. res.     object
sig.         float64
dtype: object

In [30]:
# The simplest way to do a Bonferroni correction on existing p-values is to multiply each p-value by the number of tests. 
# In this case 8:

health_res['adj. sig.'] = health_res.shape[0] * health_res['sig.']

health_res

# Note shape tells us the number of rows and columns. If we ask for index[0] it will give us the number of rows. 

,religion,race,adj. res.,sig.,adj. sig.
0,catholic,non_white,5.560118,2.695927e-08,2.156742e-07
1,catholic,white,-5.560118,2.695927e-08,2.156742e-07
2,jewish,non_white,-2.056996,3.968662e-02,3.174930e-01
3,jewish,white,2.056996,3.968662e-02,3.174930e-01
4,other,non_white,-0.258862,7.957413e-01,6.365931e+00
5,other,white,0.258862,7.957413e-01,6.365931e+00
6,protestant,non_white,-3.121115,1.801677e-03,1.441341e-02
7,protestant,white,3.121115,1.801677e-03,1.441341e-02


In [31]:
# Here using the np.round method to round the adj significance to 2 decimal places to make it easier to interpret:
health_res['ad sig new'] = np.round(health_res['adj. sig.'], 2)

health_res

,religion,race,adj. res.,sig.,adj. sig.,ad sig new
0,catholic,non_white,5.560118,2.695927e-08,2.156742e-07,0.00
1,catholic,white,-5.560118,2.695927e-08,2.156742e-07,0.00
2,jewish,non_white,-2.056996,3.968662e-02,3.174930e-01,0.32
3,jewish,white,2.056996,3.968662e-02,3.174930e-01,0.32
4,other,non_white,-0.258862,7.957413e-01,6.365931e+00,6.37
5,other,white,0.258862,7.957413e-01,6.365931e+00,6.37
6,protestant,non_white,-3.121115,1.801677e-03,1.441341e-02,0.01
7,protestant,white,3.121115,1.801677e-03,1.441341e-02,0.01
